In [3]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vikrant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
print(os.listdir('../'))
data_path = r'../data'
print(os.listdir(data_path))

['code', 'data']
['geoffery_hinton.txt', 'glove.6B.300d.txt', 'ian_godfellow.txt', 'pieter-abbeel.txt']


In [50]:
import re
text = open(os.path.join(data_path,'geoffery_hinton.txt'),'r').read()
text = re.sub('\n|>>', ' ', text).split('.')
df = pd.DataFrame(text,columns =['text'])
df.head()

,text
0,As part of this course by deeplearning
1,"ai, hope to not just teach you the technical i..."
2,The people that invented so many of these ide...
3,"In these videos, I hope to also ask these lea..."
4,"As the first of this interview series, I am d..."


In [51]:
sentences  = list(df['text'])
len(sentences)

354

In [55]:
# Extract word vectors


word_embeddings = {}
f = open(os.path.join(data_path,"glove.6B.300d.txt"), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [56]:
print('len(word_embeddings)',len(word_embeddings))

len(word_embeddings) 400000


In [57]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [58]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [59]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [60]:

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [62]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [63]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [67]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [68]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [69]:
# Extract top 10 sentences as the summary
for i in range(10):
    print(ranked_sentences[i][1])

ai, hope to not just teach you the technical ideas in deep learning, but also introduce you to some of the people, some of the heroes in deep learning
ai
As part of this course by deeplearning
 because the nice thing about ReLUs is that if you keep replicating the hidden layers and you initialize with the identity, it just copies the pattern in the layer below
 You take your measurements, and you're applying nonlinear transformations to your measurements until you get to a representation as a state vector in which the action is linear
 You shouldn't say slow
 You look at it and it just doesn't feel right
 You could do an approximate E step
 You can then do a matrix multiplier to change viewpoint, and then you can map it back to pixels
 You can give him anything and he'll come back and say, it worked
